<a href="https://colab.research.google.com/github/aabhishekdata/vid2pdf/blob/main/advanced_vid2pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytube3 moviepy fpdf pillow scikit-image


In [ ]:
pip install --upgrade pytube


In [8]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
from fpdf import FPDF
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray
import os
from PIL import Image
import numpy as np

In [9]:
# URL of the YouTube video
url = 'https://www.youtube.com/watch?v=RuRmq7VyCaQ'

# Output directory
output_dir = 'output'


In [10]:
# Download the YouTube video
try:
    yt = YouTube(url)
    stream = yt.streams.filter(file_extension='mp4', res='720p').first()
    stream.download(output_path=output_dir, filename='video.mp4')
except Exception as e:
    print(f"An error occurred: {e}")
    exit()

In [11]:
# Load the video
clip = VideoFileClip(os.path.join(output_dir, 'video.mp4'))

# SSIM threshold
ssim_threshold = 0.9

# Previous frame
prev_frame = None

# Frame counter
counter = 0

In [12]:
# Iterate through the video frame by frame
for i, frame in enumerate(clip.iter_frames(fps=1, dtype='uint8')):
    # Convert the frame to grayscale
    current_frame = rgb2gray(frame)

    # If this is not the first frame, compare with the previous frame
    if prev_frame is not None:
        s = ssim(prev_frame, current_frame)
        if s < ssim_threshold:
            frame_path = os.path.join(output_dir, f'screenshot_{counter}.png')
            im = Image.fromarray(frame)
            im.save(frame_path)
            counter += 1

    # Update the previous frame
    prev_frame = current_frame

In [13]:
# Create a PDF from the screenshots
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
for i in range(counter):
    frame_path = os.path.join(output_dir, f'screenshot_{i}.png')
    pdf.add_page()
    pdf.image(frame_path, x=10, y=10, w=190)
pdf.output(os.path.join(output_dir, 'notes.pdf'))

# Close the clip
clip.close()